##1. 라이브러리 및 데이터
##Library & Data

In [ ]:
! pip install bayesian-optimization

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from bayes_opt import BayesianOptimization
from functools import partial
import lightgbm as lgb
import warnings
import re
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


##2. 데이터 전처리
##Data Cleansing & Pre-Processing

In [ ]:
train = pd.read_csv('/content/drive/My Drive/data/train.csv')
train.info()

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [ ]:
test = pd.read_csv('/content/drive/My Drive/data/test.csv')
test.info()

In [ ]:
train_P0_ability = train[(train['event']=='Ability') & (train['player'] == 0)]
train_P1_ability = train[(train['event']=='Ability') & (train['player'] == 1)]
test_P0_ability = test[(test['event']=='Ability') & (test['player'] == 0)]
test_P1_ability = test[(test['event']=='Ability') & (test['player'] == 1)]

##3. 탐색적 자료분석
##Exploratory Data Analysis

##4. 변수 선택 및 모델 구축
##Feature Engineering & Initial Modeling

4-1. 베이스 변수(건물 변수와 동일한 방법으로 생성)

4-2. 유닛 변수(유닛 변수와 동일한 방법으로 생성)

4-3. 건물 변수

In [ ]:
x_train_build = pd.DataFrame(columns=['game_id'])
x_train_build.game_id = train.game_id.unique()

x_test_build = pd.DataFrame(columns=['game_id'])
x_test_build.game_id = test.game_id.unique()

In [ ]:
x_train_build['time'] = np.array(train[train.shift(-1).game_id != train.game_id].time)
x_train_build['time_int'] = x_train_build['time'].astype(int)
x_train_build['time_prime'] = x_train_build['time'] - x_train_build['time_int']
x_train_build['time'] = round(x_train_build['time_int'] + x_train_build['time_prime']*100/60, 2)

del x_train_build['time_int']
del x_train_build['time_prime']

x_test_build['time'] = np.array(test[test.shift(-1).game_id != test.game_id].time)
x_test_build['time_int'] = x_test_build['time'].astype(int)
x_test_build['time_prime'] = x_test_build['time'] - x_test_build['time_int']
x_test_build['time'] = round(x_test_build['time_int'] + x_test_build['time_prime']*100/60, 2)

del x_test_build['time_int']
del x_test_build['time_prime']

In [ ]:
  # winner 컬럼 추출하기 위해 불러오는 데이터
 x = pd.read_csv('/content/drive/My Drive/data/x_train_final.csv')

In [ ]:
 # winner 컬럼 추출해서 array 형태로 저장
y_train = x.winner.values
y_train

array([1, 1, 0, ..., 0, 1, 0])

In [ ]:
 # RAM 공간 확보
del x

In [ ]:
len(y_train)

38872

In [ ]:
 # 모델에 사용할 데이터
x_train_draft = pd.read_csv('/content/drive/My Drive/data/train_final_final.csv')

In [ ]:
x_train.columns

Index(['P0_species', 'P0_GetControlGroup', 'P0_Right Click', 'P1_species',
       'time', 'P1_camera_var', 'P0_right_click_var', 'P1_right_click_var',
       'game_id_P0_Ability_diff', 'game_id_P0_AddToControlGroup_diff',
       ...
       'P0_camera_var_P1_camera_var_diff',
       'P0_camera_var_P0_right_click_var_diff',
       'P0_camera_var_P1_right_click_var_diff',
       'P0_camera_var_P0_attack_count_diff',
       'P1_camera_var_P0_right_click_var_diff',
       'P1_camera_var_P1_right_click_var_diff',
       'P1_camera_var_P0_attack_count_diff',
       'P0_right_click_var_P1_attack_count_diff',
       'P1_right_click_var_P1_attack_count_diff',
       'P0_attack_count_P1_attack_count_diff'],
      dtype='object', length=1288)

In [ ]:
 # permuatation importance를 이용해서 추출한 bad_feature들 리스트로 반환
BAD_feature = ['diff_upgrade1713',
 'diff_upgrade65',
 'diff_upgrade108',
 'diff_build93',
 'P1_population_build_count',
 'diff_build69_time',
 'diff_upgrade128_time',
 'diff_unit182',
 'P1_air_defence_upgrade_count_time',
 'P1_attack_build_count',
 'diff_upgrade135',
 'diff_upgrade84',
 'diff_base1613',
 'diff_upgrade143',
 'diff_upgrade124',
 'P1_air_defence_upgrade_count',
 'P1_all_species_skill_count',
 'diff_upgrade147',
 'diff_build137',
 'P0_main_build_count',
 'diff_unit202',
 'diff_unit173',
 'diff_build1711_time',
 'P0_air_upgrade',
 'diff_upgrade155',
 'diff_upgrade1511',
 'diff_unit123',
 'diff_build93_time',
 'diff_upgrade1715_time',
 'diff_unit206',
 'diff_unit177',
 'diff_base715_time',
 'P1_ground_attack_upgrade_count',
 'P0_Complex_unit_count',
 'diff_upgrade162',
 'diff_upgrade104',
 'diff_upgrade1612',
 'diff_unit186',
 'P0_air_defence_upgrade_count',
 'diff_upgrade107',
 'diff_upgrade73',
 'diff_upgrade75',
 'diff_base75_time',
 'diff_upgrade64',
 'diff_upgrade129',
 'diff_base51',
 'P0_air_upgrade_time',
 'diff_unit2211_time',
 'diff_upgrade153_time',
 'diff_upgrade87',
 'P1_Complex_unit_count_time',
 'diff_upgrade93',
 'diff_upgrade148',
 'diff_unit84',
 'diff_upgrade159',
 'diff_unit227',
 'P0_upgrade',
 'P0_air_defence_upgrade_count_time',
 'diff_unit65',
 'P1_air_upgrade_time',
 'diff_base1210',
 'diff_upgrade119',
 'diff_upgrade142_time',
 'P1_ground_upgrade',
 'diff_base413',
 'diff_base416_time',
 'diff_upgrade126',
 'diff_upgrade43',
 'diff_unit237',
 'diff_upgrade137_time',
 'diff_unit2015',
 'diff_build177',
 'diff_build84_time',
 'P1_air_attack_upgrade_count_time',
 'diff_unit106',
 'diff_upgrade122',
 'P0_air_attack_upgrade_count_time',
 'diff_upgrade106',
 'diff_upgrade113',
 'diff_unit2210',
 'diff_build617_time',
 'diff_upgrade1410',
 'diff_upgrade129_time',
 'diff_upgrade97_time',
 'diff_base417',
 'diff_upgrade1610',
 'diff_upgrade84_time',
 'P1_ground_defence_upgrade_count',
 'diff_upgrade145',
 'diff_build164',
 'diff_build1713',
 'diff_build1411',
 'diff_upgrade146',
 'diff_base417_time',
 'diff_unit1915',
 'diff_upgrade1715',
 'diff_upgrade85',
 'diff_build146',
 'diff_build159',
 'diff_unit95',
 'P1_Complex_unit_count',
 'diff_upgrade167',
 'diff_unit2219',
 'diff_upgrade149',
 'diff_upgrade86_time',
 'diff_base1710_time',
 'diff_unit182_time',
 'P0_ControlGroup',
 'diff_base64_time',
 'diff_upgrade1614_time',
 'diff_upgrade123',
 'diff_base62_time',
 'P0_ground_unit_count',
 'diff_upgrade128',
 'diff_unit1811_time',
 'diff_upgrade142',
 'diff_unit1913_time',
 'diff_upgrade1413',
 'diff_upgrade42',
 'P0_ground_upgrade',
 'diff_unit122_time',
 'diff_upgrade147_time',
 'diff_unit2014',
 'diff_build97',
 'diff_upgrade144',
 'diff_upgrade115',
 'P1_camera_var',
 'diff_upgrade137',
 'diff_upgrade153',
 'diff_unit1715',
 'diff_unit137',
 'diff_upgrade148_time',
 'diff_unit139_time',
 'diff_unit2015_time',
 'P0_Selection_time',
 'P0_Complex_unit_count_time',
 'P0_ground_defence_upgrade_count',
 'diff_upgrade83',
 'diff_base42',
 'diff_upgrade97',
 'diff_upgrade63',
 'P1_main_build_count',
 'diff_upgrade1612_time',
 'diff_upgrade177',
 'diff_upgrade179',
 'P0_all_species_skill_count',
 'P0_ground_attack_upgrade_count',
 'diff_unit157',
 'P0_air_attack_upgrade_count',
 'P1_ControlGroup',
 'diff_upgrade164',
 'diff_unit127',
 'diff_unit147',
 'diff_upgrade82',
 'P1_C_unit_count',
 'diff_upgrade86',
 'P1_ground_unit_count',
 'diff_upgrade163',
 'diff_unit233_time',
 'diff_build613',
 'diff_unit73',
 'diff_unit87_time',
 'P1_D_unit_count',
 'diff_upgrade139_time',
 'diff_upgrade135_time',
 'diff_upgrade1311',
 'diff_unit1610_time',
 'diff_upgrade168',
 'diff_build65',
 'P1_upgrade_related_build_count',
 'diff_upgrade126_time',
 'diff_unit159',
 'diff_upgrade53',
 'diff_upgrade166',
 'P1_air_attack_upgrade_count',
 'diff_build415',
 'diff_build1015',
 'P0_C_unit_count',
 'diff_upgrade139',
 'diff_unit166',
 'P1_air_upgrade',
 'diff_upgrade1614',
 'diff_build1612',
 'P0_attack_build_count_time',
 'diff_unit167']

In [ ]:
 # unnamed 컬럼 삭제
x_train_draft = x_train_draft.drop(x_train_draft.columns[[0]], axis='columns')

In [ ]:
x_train = x_train_draft.copy()
x_train

,P0_species,P0_GetControlGroup,P0_Right Click,P1_species,time,P1_camera_var,P0_right_click_var,P1_right_click_var,game_id_P0_Ability_diff,game_id_P0_AddToControlGroup_diff,game_id_P0_Camera_diff,game_id_P0_ControlGroup_diff,game_id_P0_GetControlGroup_diff,game_id_P0_Right Click_diff,game_id_P0_Selection_diff,game_id_P0_SetControlGroup_diff,game_id_P1_Ability_diff,game_id_P1_AddToControlGroup_diff,game_id_P1_Camera_diff,game_id_P1_ControlGroup_diff,game_id_P1_GetControlGroup_diff,game_id_P1_Right Click_diff,game_id_P1_Selection_diff,game_id_P1_SetControlGroup_diff,game_id_time_diff,game_id_P0_total_unit_count_diff,game_id_P1_total_unit_count_diff,game_id_P0_worker_unit_count_diff,game_id_P1_worker_unit_count_diff,game_id_P0_Complex_unit_count_diff,game_id_P1_Complex_unit_count_diff,game_id_P0_A_unit_count_diff,game_id_P1_A_unit_count_diff,game_id_P0_B_unit_count_diff,game_id_P1_B_unit_count_diff,game_id_P0_C_unit_count_diff,game_id_P1_C_unit_count_diff,game_id_P0_D_unit_count_diff,game_id_P1_D_unit_count_diff,game_id_P0_air_unit_count_diff,...,P0_ground_attack_upgrade_count_P1_attack_count_diff,P1_ground_attack_upgrade_count_P0_all_species_skill_count_diff,P1_ground_attack_upgrade_count_P0_upgrade_diff,P1_ground_attack_upgrade_count_P0_attack_count_diff,P1_ground_attack_upgrade_count_P1_attack_count_diff,P0_ground_defence_upgrade_count_P1_all_species_skill_count_diff,P0_ground_defence_upgrade_count_P1_air_upgrade_diff,P0_ground_defence_upgrade_count_P1_upgrade_diff,P0_ground_defence_upgrade_count_P0_attack_count_diff,P1_ground_defence_upgrade_count_P1_air_upgrade_diff,P1_ground_defence_upgrade_count_P0_upgrade_diff,P1_ground_defence_upgrade_count_P1_attack_count_diff,P1_air_attack_upgrade_count_P1_all_species_skill_count_diff,P1_air_attack_upgrade_count_P1_attack_count_diff,P1_air_defence_upgrade_count_P1_all_species_skill_count_diff,P0_all_species_skill_count_P1_all_species_skill_count_diff,P0_all_species_skill_count_P1_ground_upgrade_diff,P0_all_species_skill_count_P1_upgrade_diff,P0_all_species_skill_count_P1_camera_var_diff,P0_all_species_skill_count_P1_attack_count_diff,P1_all_species_skill_count_P0_ground_upgrade_diff,P1_all_species_skill_count_P0_upgrade_diff,P1_air_upgrade_P1_attack_count_diff,P0_ground_upgrade_P1_upgrade_diff,P0_ground_upgrade_P1_attack_count_diff,P1_ground_upgrade_P0_upgrade_diff,P1_ground_upgrade_P1_attack_count_diff,P0_upgrade_P1_upgrade_diff,P1_upgrade_P0_camera_var_diff,P1_upgrade_P1_attack_count_diff,P0_camera_var_P1_camera_var_diff,P0_camera_var_P0_right_click_var_diff,P0_camera_var_P1_right_click_var_diff,P0_camera_var_P0_attack_count_diff,P1_camera_var_P0_right_click_var_diff,P1_camera_var_P1_right_click_var_diff,P1_camera_var_P0_attack_count_diff,P0_right_click_var_P1_attack_count_diff,P1_right_click_var_P1_attack_count_diff,P0_attack_count_P1_attack_count_diff
0,0,24.0,35.0,0,7.40,44.0,2367.0,77.0,-34.0,-2.0,-444.0,0.0,-24.0,-35.0,-50.0,-3.0,-34.0,0.0,-425.0,0.0,-3.0,-28.0,-57.0,-1.0,-7.40,-13,-13,-9,-6,0,0,-9,-6,-3,-5,-1,-2,0,0,-1,...,-3,0,-1,0,-3,1,1,1,1,0,-1,-3,0,-3,0,0,0,0,-44.0,-3,-1,-1,-3,1,-2,-1,-3,1,-887.0,-3,843.0,-1480.0,810.0,887.0,-2323.0,-33.0,44.0,2364.0,74.0,-3
1,1,162.0,160.0,0,9.98,1821.0,2099.0,2230.0,-76.0,0.0,-626.0,1.0,-161.0,-159.0,-185.0,-9.0,-66.0,1.0,-857.0,1.0,-30.0,-130.0,-115.0,-7.0,-8.98,-28,-27,-21,-16,1,1,-21,-16,-2,-6,-1,-3,-1,1,0,...,-3,-2,0,-1,-3,-1,0,-1,-2,0,-1,-4,-1,-4,-1,2,2,2,-1818.0,-1,0,0,-4,0,-3,0,-3,0,-749.0,-3,-1071.0,-1349.0,-1480.0,748.0,-278.0,-409.0,1819.0,2095.0,2226.0,-2
2,1,99.0,160.0,2,9.98,889.0,1250.0,1505.0,-67.0,-4.0,-411.0,2.0,-97.0,-158.0,-88.0,-12.0,-83.0,-3.0,-723.0,0.0,-107.0,-202.0,-230.0,-7.0,-7.98,-22,-37,-17,-21,2,2,-17,-28,-1,-7,0,2,2,2,-1,...,-6,0,-1,-10,-6,-2,0,-2,-11,1,-1,-6,-2,-7,-2,-1,-1,-1,-888.0,-6,1,0,-7,-1,-6,0,-5,0,-967.0,-5,80.0,-281.0,-536.0,958.0,-361.0,-616.0,878.0,1243.0,1498.0,4
3,0,132.0,276.0,1,9.98,1303.0,1731.0,1479.0,-79.0,3.0,-710.0,3.0,-129.0,-273.0,-177.0,-3.0,-86.0,3.0,-385.0,3.0,-707.0,-265.0,-145.0,-1

In [ ]:
 # bad_feature 제거
train_columns = [c for c in x_train_draft.columns if c not in BAD_feature]
x_train=x_train_draft[train_columns]
x_train.head()

,P0_Ability_time,P0_AddToControlGroup_time,P0_Camera_time,P0_ControlGroup_time,P0_GetControlGroup_time,P0_Right Click_time,P0_SetControlGroup_time,P1_Ability_time,P1_AddToControlGroup_time,P1_Camera_time,P1_ControlGroup_time,P1_GetControlGroup_time,P1_Right Click_time,P1_Selection_time,P1_SetControlGroup_time,diff_base110_time,diff_base111_time,diff_base112_time,diff_base113_time,diff_base114_time,diff_base115_time,diff_base116_time,diff_base117_time,diff_base210_time,diff_base211_time,diff_base212_time,diff_base213_time,diff_base214_time,diff_base215_time,diff_base216_time,diff_base217_time,diff_base310_time,diff_base311_time,diff_base312_time,diff_base313_time,diff_base314_time,diff_base315_time,diff_base316_time,diff_base317_time,diff_base410_time,...,diff_build1511,diff_build1513,diff_build173,diff_build175,diff_build179,diff_build1711,diff_build1715,P1_upgrade,diff_upgrade103,diff_upgrade105,diff_upgrade109,diff_upgrade125,diff_upgrade127,diff_upgrade1211,diff_upgrade1411,diff_upgrade165,diff_upgrade169,diff_upgrade1611,diff_upgrade1613,diff_upgrade1615,diff_upgrade62,diff_upgrade102,diff_upgrade1210,diff_upgrade1412,diff_upgrade95,diff_upgrade117,diff_upgrade133,diff_upgrade157,diff_upgrade1513,diff_upgrade173,diff_upgrade175,diff_upgrade1711,P0_camera_var,P0_right_click_var,P1_right_click_var,P0_attack_count,P1_attack_count,diff_camera_var,diff_right_click_var,diff_attack_count
0,4.594595,0.270270,60.000000,0.0,3.243243,4.729730,0.405405,4.594595,0.000000,57.432432,0.000000,0.405405,3.783784,7.702703,0.135135,0.000000,4.594595,-52.837838,4.594595,4.189189,0.810811,-3.108108,4.459459,-4.324324,0.270270,-57.162162,0.270270,-0.135135,-3.513514,-7.432432,0.135135,55.405405,60.000000,2.567568,60.000000,59.594595,56.216216,52.297297,59.864865,-4.594595,...,-4,-4,-9,0,-3,-4,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,887.0,2367.0,77.0,0,3,843.0,2290.0,-3
1,7.715431,0.100200,62.825651,0.0,16.232465,16.032064,1.002004,6.713427,0.000000,85.971944,0.000000,3.106212,13.126253,11.623246,0.801603,1.002004,7.715431,-78.256513,7.715431,4.609218,-5.410822,-3.907816,6.913828,-6.613226,0.100200,-85.871743,0.100200,-3.006012,-13.026052,-11.523046,-0.701403,56.112224,62.825651,-23.146293,62.825651,59.719439,49.699399,51.202405,62.024048,-6.713427,...,-8,-2,-15,1,1,-5,3,1,2,3,3,0,0,-1,0,1,1,0,1,0,-1,2,-3,1,0,1,-1,1,1,0,1,0,750.0,2099.0,2230.0,2,4,-1071.0,-131.0,-2
2,6.913828,0.601202,41.382766,0.0,9.919840,16.032064,1.402806,8.517034,0.501002,72.645291,0.200401,10.921844,20.440882,23.246493,0.901804,-1.603206,6.412826,-65.731463,6.713427,-4.008016,-13.527054,-16.332665,6.012024,-7.915832,0.100200,-72.044088,0.400802,-10.320641,-19.839679,-22.645291,-0.300601,32.865731,40.881764,-31.262525,41.182365,30.460922,20.941884,18.136273,40.480962,-8.517034,...,-3,-3,-13,0,-5,-1,2,2,0,0,1,0,1,-1,-1,1,2,0,2,0,0,0,0,0,-1,2,-1,2,2,1,1,0,969.0,1250.0,1505.0,11,7,80.0,-255.0,4
3,8.216433,0.000000,71.442886,0.0,13.226453,27.655311,0.601202,8.917836,0.000000,38.877756,0.000000,71.142285,26.853707,14.829659,1.903808,-0.701403,8.216433,-30.661323,8.216433,-62.925852,-18.637275,-6.613226,6.312625,-8.917836,0.000000,-38.877756,0.000000,-71.142285,-26.853707,-14.829659,-1.903808,62.525050,71.442886,32.565130,71.442886,0.300601,44.589178,56.613226,69.539078,-8.917836,...,-5,-3,-20,-1,-7,-5,0,1,1,0,1,-1,0,-1,0,0,1,0,1,0,-1,0,-1,1,-1,1,0,1,1,1,0,0,1300.0,1731.0,1479.0,8,17,-3.0,252.0,-9
4,9.965035,0.174825,75.174825,0.0,39.160839,30.944056,1.748252,6.293706,0.699301,47.552448,0.000000,17.307692,18.531469,22.027972,1.398601,3.671329,9.265734,-37.587413,9.965035,-7.342657,-8.566434,-12.062937,8.566434,-6.118881,-0.524476,-47.377622,0.174825,-17.132867,-18.356643,-21.853147,-1.223776,68.881119,74.475524,27.622378,75.174825,57.867133,56.643357,53.146853,73.776224,-6.293706,...,-2,-2,-5,-1,-4,-2,0,0,1,1,1,0,0,-1,-1,0,0,-1,0,0,0,1,-1,0,0,1,0,0,0,0,0,-1,1366.0,2130.0,1278.0,14,2,665.0,852.0,12


In [ ]:
x_train

,P0_species,P0_GetControlGroup,P0_Right Click,P1_species,time,P1_camera_var,P0_right_click_var,P1_right_click_var,game_id_P0_Ability_diff,game_id_P0_AddToControlGroup_diff,game_id_P0_Camera_diff,game_id_P0_ControlGroup_diff,game_id_P0_GetControlGroup_diff,game_id_P0_Right Click_diff,game_id_P0_Selection_diff,game_id_P0_SetControlGroup_diff,game_id_P1_Ability_diff,game_id_P1_AddToControlGroup_diff,game_id_P1_Camera_diff,game_id_P1_ControlGroup_diff,game_id_P1_GetControlGroup_diff,game_id_P1_Right Click_diff,game_id_P1_Selection_diff,game_id_P1_SetControlGroup_diff,game_id_time_diff,game_id_P0_total_unit_count_diff,game_id_P1_total_unit_count_diff,game_id_P0_worker_unit_count_diff,game_id_P1_worker_unit_count_diff,game_id_P0_Complex_unit_count_diff,game_id_P1_Complex_unit_count_diff,game_id_P0_A_unit_count_diff,game_id_P1_A_unit_count_diff,game_id_P0_B_unit_count_diff,game_id_P1_B_unit_count_diff,game_id_P0_C_unit_count_diff,game_id_P1_C_unit_count_diff,game_id_P0_D_unit_count_diff,game_id_P1_D_unit_count_diff,game_id_P0_air_unit_count_diff,...,P0_ground_attack_upgrade_count_P1_attack_count_diff,P1_ground_attack_upgrade_count_P0_all_species_skill_count_diff,P1_ground_attack_upgrade_count_P0_upgrade_diff,P1_ground_attack_upgrade_count_P0_attack_count_diff,P1_ground_attack_upgrade_count_P1_attack_count_diff,P0_ground_defence_upgrade_count_P1_all_species_skill_count_diff,P0_ground_defence_upgrade_count_P1_air_upgrade_diff,P0_ground_defence_upgrade_count_P1_upgrade_diff,P0_ground_defence_upgrade_count_P0_attack_count_diff,P1_ground_defence_upgrade_count_P1_air_upgrade_diff,P1_ground_defence_upgrade_count_P0_upgrade_diff,P1_ground_defence_upgrade_count_P1_attack_count_diff,P1_air_attack_upgrade_count_P1_all_species_skill_count_diff,P1_air_attack_upgrade_count_P1_attack_count_diff,P1_air_defence_upgrade_count_P1_all_species_skill_count_diff,P0_all_species_skill_count_P1_all_species_skill_count_diff,P0_all_species_skill_count_P1_ground_upgrade_diff,P0_all_species_skill_count_P1_upgrade_diff,P0_all_species_skill_count_P1_camera_var_diff,P0_all_species_skill_count_P1_attack_count_diff,P1_all_species_skill_count_P0_ground_upgrade_diff,P1_all_species_skill_count_P0_upgrade_diff,P1_air_upgrade_P1_attack_count_diff,P0_ground_upgrade_P1_upgrade_diff,P0_ground_upgrade_P1_attack_count_diff,P1_ground_upgrade_P0_upgrade_diff,P1_ground_upgrade_P1_attack_count_diff,P0_upgrade_P1_upgrade_diff,P1_upgrade_P0_camera_var_diff,P1_upgrade_P1_attack_count_diff,P0_camera_var_P1_camera_var_diff,P0_camera_var_P0_right_click_var_diff,P0_camera_var_P1_right_click_var_diff,P0_camera_var_P0_attack_count_diff,P1_camera_var_P0_right_click_var_diff,P1_camera_var_P1_right_click_var_diff,P1_camera_var_P0_attack_count_diff,P0_right_click_var_P1_attack_count_diff,P1_right_click_var_P1_attack_count_diff,P0_attack_count_P1_attack_count_diff
0,0,24.0,35.0,0,7.40,44.0,2367.0,77.0,-34.0,-2.0,-444.0,0.0,-24.0,-35.0,-50.0,-3.0,-34.0,0.0,-425.0,0.0,-3.0,-28.0,-57.0,-1.0,-7.40,-13,-13,-9,-6,0,0,-9,-6,-3,-5,-1,-2,0,0,-1,...,-3,0,-1,0,-3,1,1,1,1,0,-1,-3,0,-3,0,0,0,0,-44.0,-3,-1,-1,-3,1,-2,-1,-3,1,-887.0,-3,843.0,-1480.0,810.0,887.0,-2323.0,-33.0,44.0,2364.0,74.0,-3
1,1,162.0,160.0,0,9.98,1821.0,2099.0,2230.0,-76.0,0.0,-626.0,1.0,-161.0,-159.0,-185.0,-9.0,-66.0,1.0,-857.0,1.0,-30.0,-130.0,-115.0,-7.0,-8.98,-28,-27,-21,-16,1,1,-21,-16,-2,-6,-1,-3,-1,1,0,...,-3,-2,0,-1,-3,-1,0,-1,-2,0,-1,-4,-1,-4,-1,2,2,2,-1818.0,-1,0,0,-4,0,-3,0,-3,0,-749.0,-3,-1071.0,-1349.0,-1480.0,748.0,-278.0,-409.0,1819.0,2095.0,2226.0,-2
2,1,99.0,160.0,2,9.98,889.0,1250.0,1505.0,-67.0,-4.0,-411.0,2.0,-97.0,-158.0,-88.0,-12.0,-83.0,-3.0,-723.0,0.0,-107.0,-202.0,-230.0,-7.0,-7.98,-22,-37,-17,-21,2,2,-17,-28,-1,-7,0,2,2,2,-1,...,-6,0,-1,-10,-6,-2,0,-2,-11,1,-1,-6,-2,-7,-2,-1,-1,-1,-888.0,-6,1,0,-7,-1,-6,0,-5,0,-967.0,-5,80.0,-281.0,-536.0,958.0,-361.0,-616.0,878.0,1243.0,1498.0,4
3,0,132.0,276.0,1,9.98,1303.0,1731.0,1479.0,-79.0,3.0,-710.0,3.0,-129.0,-273.0,-177.0,-3.0,-86.0,3.0,-385.0,3.0,-707.0,-265.0,-145.0,-1

In [ ]:
x_train.head()

,P0_Ability_time,P0_AddToControlGroup_time,P0_Camera_time,P0_ControlGroup_time,P0_GetControlGroup_time,P0_Right Click_time,P0_Selection_time,P0_SetControlGroup_time,P1_Ability_time,P1_AddToControlGroup_time,P1_Camera_time,P1_ControlGroup_time,P1_GetControlGroup_time,P1_Right Click_time,P1_Selection_time,P1_SetControlGroup_time,diff_base110_time,diff_base111_time,diff_base112_time,diff_base113_time,diff_base114_time,diff_base115_time,diff_base116_time,diff_base117_time,diff_base210_time,diff_base211_time,diff_base212_time,diff_base213_time,diff_base214_time,diff_base215_time,diff_base216_time,diff_base217_time,diff_base310_time,diff_base311_time,diff_base312_time,diff_base313_time,diff_base314_time,diff_base315_time,diff_base316_time,diff_base317_time,...,diff_upgrade1610,diff_upgrade1612,diff_upgrade1614,diff_upgrade53,diff_upgrade73,diff_upgrade75,diff_upgrade93,diff_upgrade95,diff_upgrade97,diff_upgrade113,diff_upgrade115,diff_upgrade117,diff_upgrade119,diff_upgrade133,diff_upgrade135,diff_upgrade137,diff_upgrade139,diff_upgrade1311,diff_upgrade153,diff_upgrade155,diff_upgrade157,diff_upgrade159,diff_upgrade1511,diff_upgrade1513,diff_upgrade173,diff_upgrade175,diff_upgrade177,diff_upgrade179,diff_upgrade1711,diff_upgrade1713,diff_upgrade1715,P0_camera_var,P1_camera_var,P0_right_click_var,P1_right_click_var,P0_attack_count,P1_attack_count,diff_camera_var,diff_right_click_var,diff_attack_count
0,4.594595,0.270270,60.000000,0.0,3.243243,4.729730,6.756757,0.405405,4.594595,0.000000,57.432432,0.000000,0.405405,3.783784,7.702703,0.135135,0.000000,4.594595,-52.837838,4.594595,4.189189,0.810811,-3.108108,4.459459,-4.324324,0.270270,-57.162162,0.270270,-0.135135,-3.513514,-7.432432,0.135135,55.405405,60.000000,2.567568,60.000000,59.594595,56.216216,52.297297,59.864865,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,887.0,44.0,2367.0,77.0,0,3,843.0,2290.0,-3
1,7.715431,0.100200,62.825651,0.0,16.232465,16.032064,18.637275,1.002004,6.713427,0.000000,85.971944,0.000000,3.106212,13.126253,11.623246,0.801603,1.002004,7.715431,-78.256513,7.715431,4.609218,-5.410822,-3.907816,6.913828,-6.613226,0.100200,-85.871743,0.100200,-3.006012,-13.026052,-11.523046,-0.701403,56.112224,62.825651,-23.146293,62.825651,59.719439,49.699399,51.202405,62.024048,...,-2,1,0,-1,-1,0,-1,0,0,0,1,1,1,-1,0,0,0,-1,0,1,1,1,0,1,0,1,1,1,0,1,0,750.0,1821.0,2099.0,2230.0,2,4,-1071.0,-131.0,-2
2,6.913828,0.601202,41.382766,0.0,9.919840,16.032064,9.018036,1.402806,8.517034,0.501002,72.645291,0.200401,10.921844,20.440882,23.246493,0.901804,-1.603206,6.412826,-65.731463,6.713427,-4.008016,-13.527054,-16.332665,6.012024,-7.915832,0.100200,-72.044088,0.400802,-10.320641,-19.839679,-22.645291,-0.300601,32.865731,40.881764,-31.262525,41.182365,30.460922,20.941884,18.136273,40.480962,...,1,1,1,0,-1,-1,-1,-1,0,1,1,2,2,-1,-1,0,0,-2,1,1,2,2,0,2,1,1,2,2,0,2,0,969.0,889.0,1250.0,1505.0,11,7,80.0,-255.0,4
3,8.216433,0.000000,71.442886,0.0,13.226453,27.655311,18.036072,0.601202,8.917836,0.000000,38.877756,0.000000,71.142285,26.853707,14.829659,1.903808,-0.701403,8.216433,-30.661323,8.216433,-62.925852,-18.637275,-6.613226,6.312625,-8.917836,0.000000,-38.877756,0.000000,-71.142285,-26.853707,-14.829659,-1.903808,62.525050,71.442886,32.565130,71.442886,0.300601,44.589178,56.613226,69.539078,...,0,1,0,1,0,-1,0,-1,0,1,0,1,1,0,-1,0,0,-1,1,0,1,1,0,1,1,0,1,1,0,1,0,1300.0,1303.0,1731.0,1479.0,8,17,-3.0,252.0,-9
4,9.965035,0.174825,75.174825,0.0,39.160839,30.944056,11.713287,1.748252,6.293706,0.699301,47.552448,0.000000,17.307692,18.531469,22.027972,1.398601,3.671329,9.265734,-37.587413,9.965035,-7.342657,-8.566434,-12.062937,8.566434,-6.118881,-0.524476,-47.377622,0.174825,-17.132867,-18.356643,-21.853147,-1.223776,68.881119,74.475524,27.622378,75.174825,57.867133,56.643357,53.146853,73.776224,...,-1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1366.0,701.0,2130.0,1278.0,14,2,665.0,852.0,12


In [ ]:
y_train

array([1, 1, 0, ..., 0, 1, 0])

In [ ]:
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, min_child_weight, boosting_type, metric, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    kf = KFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
        model = lgb.LGBMClassifier(
            num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
            min_child_weight = min_child_weight,
            boosting_type = 'dart',
            metric = 'auc'
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [ ]:
# 모델과 관련없는 변수 고정
func_fixed = partial(lgb_cv, x_data=x_train, y_data=y_train, n_splits=5, output='score', boosting_type = 'dart', metric='auc') 
# 베이지안 최적화 범위 설정
lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (16, 4096),        # num_leaves,       범위(16~4096)
        'learning_rate': (0.0001, 0.07),  # learning_rate,   범위(0.0001~0.07)
        'n_estimators': (16, 4096),      # n_estimators,     범위(16~4096)
        'subsample': (0, 1),             # subsample,        범위(0~1)
        'colsample_bytree': (0, 1),      # colsample_bytree, 범위(0~1)
        'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),           # reg_lambda,       범위(0~50)
        'min_child_weight': (0, 100),      # min_child_weight, 범위(0~1)
    }, 
    random_state=1005                  # 시드 고정
)
lgbBO.maximize(init_points=10, n_iter=100)

|   iter    |  target   | colsam... | learni... | min_ch... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6983   |  0.6592   |  0.005049 |  47.05    |  3.455e+0 |  730.3    |  2.122    |  10.89    |  0.03891  |
|  2        |  0.6951   |  0.7203   |  0.03116  |  34.05    |  3.575e+0 |  2.443e+0 |  4.366    |  40.42    |  0.4016   |
|  3        |  0.6983   |  0.2682   |  0.02503  |  64.88    |  3.47e+03 |  732.3    |  3.471    |  33.97    |  0.7474   |
|  4        |  0.6948   |  0.0253   |  0.05906  |  59.34    |  2.615e+0 |  394.9    |  7.376    |  29.31    |  0.7373   |
|  5        |  0.6985   |  0.5848   |  0.0287   |  35.96    |  513.5    |  595.7    |  1.667    |  4.671    |  0.8535   |
|  6        |  0.6901   |  0.8257   |  0.05121  |  43.76    |  3.775e+0 |  2.352e+0 |  4.362    |  14.37    |  0.7397   |
|  7        |  0.6924   

KeyboardInterrupt: ignored

In [ ]:
params = lgbBO.max['params']
models = lgb_cv(
    params['num_leaves'], 
    params['learning_rate'], 
    params['n_estimators'], 
    params['subsample'], 
    params['colsample_bytree'], 
    params['reg_alpha'], 
    params['reg_lambda'],
    params['min_child_weight'],
    x_data=x_train, y_data=y_train, n_splits=5, output='model', boosting_type='dart', metric='auc')

In [ ]:
x_test = pd.read_csv('/content/drive/My Drive/data/real_test.csv')

In [ ]:
x_test

,Unnamed: 0,P0_species,P0_GetControlGroup,P0_Right Click,P1_species,time,P1_camera_var,P0_right_click_var,P1_right_click_var,game_id_P0_Ability_diff,game_id_P0_AddToControlGroup_diff,game_id_P0_Camera_diff,game_id_P0_ControlGroup_diff,game_id_P0_GetControlGroup_diff,game_id_P0_Right Click_diff,game_id_P0_Selection_diff,game_id_P0_SetControlGroup_diff,game_id_P1_Ability_diff,game_id_P1_AddToControlGroup_diff,game_id_P1_Camera_diff,game_id_P1_ControlGroup_diff,game_id_P1_GetControlGroup_diff,game_id_P1_Right Click_diff,game_id_P1_Selection_diff,game_id_P1_SetControlGroup_diff,game_id_time_diff,game_id_P0_total_unit_count_diff,game_id_P1_total_unit_count_diff,game_id_P0_worker_unit_count_diff,game_id_P1_worker_unit_count_diff,game_id_P0_Complex_unit_count_diff,game_id_P1_Complex_unit_count_diff,game_id_P0_A_unit_count_diff,game_id_P1_A_unit_count_diff,game_id_P0_B_unit_count_diff,game_id_P1_B_unit_count_diff,game_id_P0_C_unit_count_diff,game_id_P1_C_unit_count_diff,game_id_P0_D_unit_count_diff,game_id_P1_D_unit_count_diff,...,P0_ground_attack_upgrade_count_P1_attack_count_diff,P1_ground_attack_upgrade_count_P0_all_species_skill_count_diff,P1_ground_attack_upgrade_count_P0_upgrade_diff,P1_ground_attack_upgrade_count_P0_attack_count_diff,P1_ground_attack_upgrade_count_P1_attack_count_diff,P0_ground_defence_upgrade_count_P1_all_species_skill_count_diff,P0_ground_defence_upgrade_count_P1_air_upgrade_diff,P0_ground_defence_upgrade_count_P1_upgrade_diff,P0_ground_defence_upgrade_count_P0_attack_count_diff,P1_ground_defence_upgrade_count_P1_air_upgrade_diff,P1_ground_defence_upgrade_count_P0_upgrade_diff,P1_ground_defence_upgrade_count_P1_attack_count_diff,P1_air_attack_upgrade_count_P1_all_species_skill_count_diff,P1_air_attack_upgrade_count_P1_attack_count_diff,P1_air_defence_upgrade_count_P1_all_species_skill_count_diff,P0_all_species_skill_count_P1_all_species_skill_count_diff,P0_all_species_skill_count_P1_ground_upgrade_diff,P0_all_species_skill_count_P1_upgrade_diff,P0_all_species_skill_count_P1_camera_var_diff,P0_all_species_skill_count_P1_attack_count_diff,P1_all_species_skill_count_P0_ground_upgrade_diff,P1_all_species_skill_count_P0_upgrade_diff,P1_air_upgrade_P1_attack_count_diff,P0_ground_upgrade_P1_upgrade_diff,P0_ground_upgrade_P1_attack_count_diff,P1_ground_upgrade_P0_upgrade_diff,P1_ground_upgrade_P1_attack_count_diff,P0_upgrade_P1_upgrade_diff,P1_upgrade_P0_camera_var_diff,P1_upgrade_P1_attack_count_diff,P0_camera_var_P1_camera_var_diff,P0_camera_var_P0_right_click_var_diff,P0_camera_var_P1_right_click_var_diff,P0_camera_var_P0_attack_count_diff,P1_camera_var_P0_right_click_var_diff,P1_camera_var_P1_right_click_var_diff,P1_camera_var_P0_attack_count_diff,P0_right_click_var_P1_attack_count_diff,P1_right_click_var_P1_attack_count_diff,P0_attack_count_P1_attack_count_diff
0,0,1,9.0,241.0,1,7.10,1595.0,1312.0,1529.0,38841.0,38871.0,38640.0,38872.0,38863.0,38631.0,38820.0,38867.0,38834.0,38868.0,38405.0,38872.0,38829.0,38750.0,38831.0,38872.0,38864.90,38859,38856,38861,38860,38872,38872,38861,38860,38861,38860,38872,38872,38872,38872,...,-2,-1,0,-5,-2,-1,0,0,-5,0,0,-2,-1,-2,-1,0,1,1,-1594.0,-1,1,1,-2,0,-2,0,-2,0,-635.0,-2,-960.0,-677.0,-894.0,630.0,283.0,66.0,1590.0,1310.0,1527.0,3
1,1,2,18.0,51.0,1,6.12,1028.0,841.0,1744.0,38839.0,38873.0,38673.0,38869.0,38855.0,38822.0,38759.0,38873.0,38845.0,38872.0,38727.0,38873.0,38751.0,38805.0,38854.0,38868.0,38866.88,38855,38856,38866,38860,38873,38873,38860,38860,38860,38860,38873,38873,38873,38873,...,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,-1027.0,1,0,0,0,0,0,0,0,0,-984.0,0,-44.0,143.0,-760.0,984.0,187.0,-716.0,1028.0,841.0,1744.0,0
2,2,0,63.0,79.0,0,4.68,451.0,2140.0,640.0,38841.0,38874.0,38629.0,38874.0,38811.0,38795.0,38832.0,38867.0,38845.0,38874.0,38654.0,38874.0,38448.0,38766.0,38838.0,38869.0,38869.32,38862,38867,38865,38869,38874,38874,38865,38869,38865,38869,38873,38872,38874,38874,...,-4,0,0,-9,-4,0,0,0,-9,0,0,-4,0,-4,0,0,0,0,-451.0,-4,0,0,-4,0,-4,0,-4,0,-1739.0,-4,1288.0,-

In [ ]:
del x_test['Unnamed: 0']

In [ ]:
x_test

,P0_species,P0_GetControlGroup,P0_Right Click,P1_species,time,P1_camera_var,P0_right_click_var,P1_right_click_var,game_id_P0_Ability_diff,game_id_P0_AddToControlGroup_diff,game_id_P0_Camera_diff,game_id_P0_ControlGroup_diff,game_id_P0_GetControlGroup_diff,game_id_P0_Right Click_diff,game_id_P0_Selection_diff,game_id_P0_SetControlGroup_diff,game_id_P1_Ability_diff,game_id_P1_AddToControlGroup_diff,game_id_P1_Camera_diff,game_id_P1_ControlGroup_diff,game_id_P1_GetControlGroup_diff,game_id_P1_Right Click_diff,game_id_P1_Selection_diff,game_id_P1_SetControlGroup_diff,game_id_time_diff,game_id_P0_total_unit_count_diff,game_id_P1_total_unit_count_diff,game_id_P0_worker_unit_count_diff,game_id_P1_worker_unit_count_diff,game_id_P0_Complex_unit_count_diff,game_id_P1_Complex_unit_count_diff,game_id_P0_A_unit_count_diff,game_id_P1_A_unit_count_diff,game_id_P0_B_unit_count_diff,game_id_P1_B_unit_count_diff,game_id_P0_C_unit_count_diff,game_id_P1_C_unit_count_diff,game_id_P0_D_unit_count_diff,game_id_P1_D_unit_count_diff,game_id_P0_air_unit_count_diff,...,P0_ground_attack_upgrade_count_P1_attack_count_diff,P1_ground_attack_upgrade_count_P0_all_species_skill_count_diff,P1_ground_attack_upgrade_count_P0_upgrade_diff,P1_ground_attack_upgrade_count_P0_attack_count_diff,P1_ground_attack_upgrade_count_P1_attack_count_diff,P0_ground_defence_upgrade_count_P1_all_species_skill_count_diff,P0_ground_defence_upgrade_count_P1_air_upgrade_diff,P0_ground_defence_upgrade_count_P1_upgrade_diff,P0_ground_defence_upgrade_count_P0_attack_count_diff,P1_ground_defence_upgrade_count_P1_air_upgrade_diff,P1_ground_defence_upgrade_count_P0_upgrade_diff,P1_ground_defence_upgrade_count_P1_attack_count_diff,P1_air_attack_upgrade_count_P1_all_species_skill_count_diff,P1_air_attack_upgrade_count_P1_attack_count_diff,P1_air_defence_upgrade_count_P1_all_species_skill_count_diff,P0_all_species_skill_count_P1_all_species_skill_count_diff,P0_all_species_skill_count_P1_ground_upgrade_diff,P0_all_species_skill_count_P1_upgrade_diff,P0_all_species_skill_count_P1_camera_var_diff,P0_all_species_skill_count_P1_attack_count_diff,P1_all_species_skill_count_P0_ground_upgrade_diff,P1_all_species_skill_count_P0_upgrade_diff,P1_air_upgrade_P1_attack_count_diff,P0_ground_upgrade_P1_upgrade_diff,P0_ground_upgrade_P1_attack_count_diff,P1_ground_upgrade_P0_upgrade_diff,P1_ground_upgrade_P1_attack_count_diff,P0_upgrade_P1_upgrade_diff,P1_upgrade_P0_camera_var_diff,P1_upgrade_P1_attack_count_diff,P0_camera_var_P1_camera_var_diff,P0_camera_var_P0_right_click_var_diff,P0_camera_var_P1_right_click_var_diff,P0_camera_var_P0_attack_count_diff,P1_camera_var_P0_right_click_var_diff,P1_camera_var_P1_right_click_var_diff,P1_camera_var_P0_attack_count_diff,P0_right_click_var_P1_attack_count_diff,P1_right_click_var_P1_attack_count_diff,P0_attack_count_P1_attack_count_diff
0,1,9.0,241.0,1,7.10,1595.0,1312.0,1529.0,38841.0,38871.0,38640.0,38872.0,38863.0,38631.0,38820.0,38867.0,38834.0,38868.0,38405.0,38872.0,38829.0,38750.0,38831.0,38872.0,38864.90,38859,38856,38861,38860,38872,38872,38861,38860,38861,38860,38872,38872,38872,38872,38871,...,-2,-1,0,-5,-2,-1,0,0,-5,0,0,-2,-1,-2,-1,0,1,1,-1594.0,-1,1,1,-2,0,-2,0,-2,0,-635.0,-2,-960.0,-677.0,-894.0,630.0,283.0,66.0,1590.0,1310.0,1527.0,3
1,2,18.0,51.0,1,6.12,1028.0,841.0,1744.0,38839.0,38873.0,38673.0,38869.0,38855.0,38822.0,38759.0,38873.0,38845.0,38872.0,38727.0,38873.0,38751.0,38805.0,38854.0,38868.0,38866.88,38855,38856,38866,38860,38873,38873,38860,38860,38860,38860,38873,38873,38873,38873,38873,...,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,-1027.0,1,0,0,0,0,0,0,0,0,-984.0,0,-44.0,143.0,-760.0,984.0,187.0,-716.0,1028.0,841.0,1744.0,0
2,0,63.0,79.0,0,4.68,451.0,2140.0,640.0,38841.0,38874.0,38629.0,38874.0,38811.0,38795.0,38832.0,38867.0,38845.0,38874.0,38654.0,38874.0,38448.0,38766.0,38838.0,38869.0,38869.32,38862,38867,38865,38869,38874,38874,38865,38869,38865,38869,38873,38872,38874,38874,38873,...,-4,0,0,-9,-4,0,0,0,-9,0,0,-4,0,-4,0,0,0,0,-451.0,-4,0,0,-4

In [ ]:
x_train

,P0_species,P0_GetControlGroup,P0_Right Click,P1_species,time,P1_camera_var,P0_right_click_var,P1_right_click_var,game_id_P0_Ability_diff,game_id_P0_AddToControlGroup_diff,game_id_P0_Camera_diff,game_id_P0_ControlGroup_diff,game_id_P0_GetControlGroup_diff,game_id_P0_Right Click_diff,game_id_P0_Selection_diff,game_id_P0_SetControlGroup_diff,game_id_P1_Ability_diff,game_id_P1_AddToControlGroup_diff,game_id_P1_Camera_diff,game_id_P1_ControlGroup_diff,game_id_P1_GetControlGroup_diff,game_id_P1_Right Click_diff,game_id_P1_Selection_diff,game_id_P1_SetControlGroup_diff,game_id_time_diff,game_id_P0_total_unit_count_diff,game_id_P1_total_unit_count_diff,game_id_P0_worker_unit_count_diff,game_id_P1_worker_unit_count_diff,game_id_P0_Complex_unit_count_diff,game_id_P1_Complex_unit_count_diff,game_id_P0_A_unit_count_diff,game_id_P1_A_unit_count_diff,game_id_P0_B_unit_count_diff,game_id_P1_B_unit_count_diff,game_id_P0_C_unit_count_diff,game_id_P1_C_unit_count_diff,game_id_P0_D_unit_count_diff,game_id_P1_D_unit_count_diff,game_id_P0_air_unit_count_diff,...,P0_ground_attack_upgrade_count_P1_attack_count_diff,P1_ground_attack_upgrade_count_P0_all_species_skill_count_diff,P1_ground_attack_upgrade_count_P0_upgrade_diff,P1_ground_attack_upgrade_count_P0_attack_count_diff,P1_ground_attack_upgrade_count_P1_attack_count_diff,P0_ground_defence_upgrade_count_P1_all_species_skill_count_diff,P0_ground_defence_upgrade_count_P1_air_upgrade_diff,P0_ground_defence_upgrade_count_P1_upgrade_diff,P0_ground_defence_upgrade_count_P0_attack_count_diff,P1_ground_defence_upgrade_count_P1_air_upgrade_diff,P1_ground_defence_upgrade_count_P0_upgrade_diff,P1_ground_defence_upgrade_count_P1_attack_count_diff,P1_air_attack_upgrade_count_P1_all_species_skill_count_diff,P1_air_attack_upgrade_count_P1_attack_count_diff,P1_air_defence_upgrade_count_P1_all_species_skill_count_diff,P0_all_species_skill_count_P1_all_species_skill_count_diff,P0_all_species_skill_count_P1_ground_upgrade_diff,P0_all_species_skill_count_P1_upgrade_diff,P0_all_species_skill_count_P1_camera_var_diff,P0_all_species_skill_count_P1_attack_count_diff,P1_all_species_skill_count_P0_ground_upgrade_diff,P1_all_species_skill_count_P0_upgrade_diff,P1_air_upgrade_P1_attack_count_diff,P0_ground_upgrade_P1_upgrade_diff,P0_ground_upgrade_P1_attack_count_diff,P1_ground_upgrade_P0_upgrade_diff,P1_ground_upgrade_P1_attack_count_diff,P0_upgrade_P1_upgrade_diff,P1_upgrade_P0_camera_var_diff,P1_upgrade_P1_attack_count_diff,P0_camera_var_P1_camera_var_diff,P0_camera_var_P0_right_click_var_diff,P0_camera_var_P1_right_click_var_diff,P0_camera_var_P0_attack_count_diff,P1_camera_var_P0_right_click_var_diff,P1_camera_var_P1_right_click_var_diff,P1_camera_var_P0_attack_count_diff,P0_right_click_var_P1_attack_count_diff,P1_right_click_var_P1_attack_count_diff,P0_attack_count_P1_attack_count_diff
0,0,24.0,35.0,0,7.40,44.0,2367.0,77.0,-34.0,-2.0,-444.0,0.0,-24.0,-35.0,-50.0,-3.0,-34.0,0.0,-425.0,0.0,-3.0,-28.0,-57.0,-1.0,-7.40,-13,-13,-9,-6,0,0,-9,-6,-3,-5,-1,-2,0,0,-1,...,-3,0,-1,0,-3,1,1,1,1,0,-1,-3,0,-3,0,0,0,0,-44.0,-3,-1,-1,-3,1,-2,-1,-3,1,-887.0,-3,843.0,-1480.0,810.0,887.0,-2323.0,-33.0,44.0,2364.0,74.0,-3
1,1,162.0,160.0,0,9.98,1821.0,2099.0,2230.0,-76.0,0.0,-626.0,1.0,-161.0,-159.0,-185.0,-9.0,-66.0,1.0,-857.0,1.0,-30.0,-130.0,-115.0,-7.0,-8.98,-28,-27,-21,-16,1,1,-21,-16,-2,-6,-1,-3,-1,1,0,...,-3,-2,0,-1,-3,-1,0,-1,-2,0,-1,-4,-1,-4,-1,2,2,2,-1818.0,-1,0,0,-4,0,-3,0,-3,0,-749.0,-3,-1071.0,-1349.0,-1480.0,748.0,-278.0,-409.0,1819.0,2095.0,2226.0,-2
2,1,99.0,160.0,2,9.98,889.0,1250.0,1505.0,-67.0,-4.0,-411.0,2.0,-97.0,-158.0,-88.0,-12.0,-83.0,-3.0,-723.0,0.0,-107.0,-202.0,-230.0,-7.0,-7.98,-22,-37,-17,-21,2,2,-17,-28,-1,-7,0,2,2,2,-1,...,-6,0,-1,-10,-6,-2,0,-2,-11,1,-1,-6,-2,-7,-2,-1,-1,-1,-888.0,-6,1,0,-7,-1,-6,0,-5,0,-967.0,-5,80.0,-281.0,-536.0,958.0,-361.0,-616.0,878.0,1243.0,1498.0,4
3,0,132.0,276.0,1,9.98,1303.0,1731.0,1479.0,-79.0,3.0,-710.0,3.0,-129.0,-273.0,-177.0,-3.0,-86.0,3.0,-385.0,3.0,-707.0,-265.0,-145.0,-1

In [ ]:
del x_test['game_id']

In [ ]:
x_test

,P0_species,P0_Ability,P0_AddToControlGroup,P0_Camera,P0_ControlGroup,P0_GetControlGroup,P0_Right Click,P0_Selection,P0_SetControlGroup,P1_species,P1_Ability,P1_AddToControlGroup,P1_Camera,P1_ControlGroup,P1_GetControlGroup,P1_Right Click,P1_Selection,P1_SetControlGroup,diff_base110,diff_base111,diff_base112,diff_base113,diff_base114,diff_base115,diff_base116,diff_base117,diff_base210,diff_base211,diff_base212,diff_base213,diff_base214,diff_base215,diff_base216,diff_base217,diff_base310,diff_base311,diff_base312,diff_base313,diff_base314,diff_base315,...,diff_upgrade1410_time,diff_upgrade1412_time,diff_upgrade162_time,diff_upgrade164_time,diff_upgrade166_time,diff_upgrade168_time,diff_upgrade1610_time,diff_upgrade1612_time,diff_upgrade1614_time,diff_upgrade53_time,diff_upgrade73_time,diff_upgrade75_time,diff_upgrade93_time,diff_upgrade95_time,diff_upgrade97_time,diff_upgrade113_time,diff_upgrade115_time,diff_upgrade117_time,diff_upgrade119_time,diff_upgrade133_time,diff_upgrade135_time,diff_upgrade137_time,diff_upgrade139_time,diff_upgrade1311_time,diff_upgrade153_time,diff_upgrade155_time,diff_upgrade157_time,diff_upgrade159_time,diff_upgrade1511_time,diff_upgrade1513_time,diff_upgrade173_time,diff_upgrade175_time,diff_upgrade177_time,diff_upgrade179_time,diff_upgrade1711_time,diff_upgrade1713_time,diff_upgrade1715_time,P0_attack_count_time,P1_attack_count_time,diff_attack_count_time
0,1,31.0,1.0,232.0,0.0,9.0,241.0,52.0,5.0,1,38.0,4.0,467.0,0.0,43.0,122.0,41.0,0.0,-7.0,27.0,-436.0,31.0,-12.0,-91.0,-10.0,31.0,-37.0,-3.0,-466.0,1.0,-42.0,-121.0,-40.0,1.0,194.0,228.0,-235.0,232.0,189.0,110.0,...,-0.140845,0.000000,0.000000,0.000000,0.000000,0.000000,-0.140845,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.140845,0.140845,0.140845,0.140845,0.000000,0.000000,0.0,0.0,-0.140845,0.000000,0.000000,0.000000,0.000000,-0.140845,0.000000,0.000000,0.000000,0.000000,0.000000,-0.140845,0.000000,0.0,0.704225,0.281690,0.422535
1,2,34.0,0.0,200.0,4.0,18.0,51.0,114.0,0.0,1,28.0,1.0,146.0,0.0,122.0,68.0,19.0,5.0,6.0,33.0,-112.0,34.0,-88.0,-34.0,15.0,29.0,-28.0,-1.0,-146.0,0.0,-122.0,-68.0,-19.0,-5.0,172.0,199.0,54.0,200.0,78.0,132.0,...,-0.163399,0.000000,0.000000,0.000000,0.000000,0.000000,-0.163399,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,0,33.0,0.0,245.0,0.0,63.0,79.0,42.0,7.0,0,29.0,0.0,220.0,0.0,426.0,108.0,36.0,5.0,4.0,33.0,-187.0,33.0,-393.0,-75.0,-3.0,28.0,-29.0,0.0,-220.0,0.0,-426.0,-108.0,-36.0,-5.0,216.0,245.0,25.0,245.0,-181.0,137.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.923077,0.854701,1.068376
3,0,95.0,8.0,515.0,0.0,255.0,218.0,123.0,9.0,1,58.0,1.0,357.0,0.0,24.0,322.0,99.0,2.0,37.0,94.0,-262.0,95.0,71.0,-227.0,-4.0,93.0,-50.0,7.0,-349.0,8.0,-16.0,-314.0,-91.0,6.0,457.0,514.0,158.0,515.0,491.0,193.0,...,0.000000,0.125471,0.000000,0.125471,0.125471,0.125471,0.000000,0.125471,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250941,0.250941,0.250941,0.250941,0.000000,0.000000,0.0,0.0,-0.250941,0.000000,0.000000,0.000000,0.000000,-0.250941,0.000000,0.000000,0.000000,0.000000,0.000000,-0.250941,0.000000,0.0,1.631117,2.132999,-0.501882
4,2,7.0,2.0,129.0,0.0,387.0,30.0,25.0,2.0,2,8.0,0.0,161.0,0.0,1.0,7.0,35.0,1.0,-1.0,7.0,-154.0,7.0,6.0,0.0,-28.0,6.0,-6.0,2.0,-159.0,2.0,1.0,-5.0,-33.0,1.0,121.0,129.0,-32.0,129.0,128.0,122.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.353357,0.353357,0.353357,0.353357,0.000000,0.000000,0.0,0.0,-0.35

In [ ]:
x_train.head()

,P0_Ability_time,P0_AddToControlGroup_time,P0_Camera_time,P0_ControlGroup_time,P0_GetControlGroup_time,P0_Right Click_time,P0_Selection_time,P0_SetControlGroup_time,P1_Ability_time,P1_AddToControlGroup_time,P1_Camera_time,P1_ControlGroup_time,P1_GetControlGroup_time,P1_Right Click_time,P1_Selection_time,P1_SetControlGroup_time,diff_base110_time,diff_base111_time,diff_base112_time,diff_base113_time,diff_base114_time,diff_base115_time,diff_base116_time,diff_base117_time,diff_base210_time,diff_base211_time,diff_base212_time,diff_base213_time,diff_base214_time,diff_base215_time,diff_base216_time,diff_base217_time,diff_base310_time,diff_base311_time,diff_base312_time,diff_base313_time,diff_base314_time,diff_base315_time,diff_base316_time,diff_base317_time,...,diff_upgrade1610,diff_upgrade1612,diff_upgrade1614,diff_upgrade53,diff_upgrade73,diff_upgrade75,diff_upgrade93,diff_upgrade95,diff_upgrade97,diff_upgrade113,diff_upgrade115,diff_upgrade117,diff_upgrade119,diff_upgrade133,diff_upgrade135,diff_upgrade137,diff_upgrade139,diff_upgrade1311,diff_upgrade153,diff_upgrade155,diff_upgrade157,diff_upgrade159,diff_upgrade1511,diff_upgrade1513,diff_upgrade173,diff_upgrade175,diff_upgrade177,diff_upgrade179,diff_upgrade1711,diff_upgrade1713,diff_upgrade1715,P0_camera_var,P1_camera_var,P0_right_click_var,P1_right_click_var,P0_attack_count,P1_attack_count,diff_camera_var,diff_right_click_var,diff_attack_count
0,4.594595,0.270270,60.000000,0.0,3.243243,4.729730,6.756757,0.405405,4.594595,0.000000,57.432432,0.000000,0.405405,3.783784,7.702703,0.135135,0.000000,4.594595,-52.837838,4.594595,4.189189,0.810811,-3.108108,4.459459,-4.324324,0.270270,-57.162162,0.270270,-0.135135,-3.513514,-7.432432,0.135135,55.405405,60.000000,2.567568,60.000000,59.594595,56.216216,52.297297,59.864865,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,887.0,44.0,2367.0,77.0,0,3,843.0,2290.0,-3
1,7.715431,0.100200,62.825651,0.0,16.232465,16.032064,18.637275,1.002004,6.713427,0.000000,85.971944,0.000000,3.106212,13.126253,11.623246,0.801603,1.002004,7.715431,-78.256513,7.715431,4.609218,-5.410822,-3.907816,6.913828,-6.613226,0.100200,-85.871743,0.100200,-3.006012,-13.026052,-11.523046,-0.701403,56.112224,62.825651,-23.146293,62.825651,59.719439,49.699399,51.202405,62.024048,...,-2,1,0,-1,-1,0,-1,0,0,0,1,1,1,-1,0,0,0,-1,0,1,1,1,0,1,0,1,1,1,0,1,0,750.0,1821.0,2099.0,2230.0,2,4,-1071.0,-131.0,-2
2,6.913828,0.601202,41.382766,0.0,9.919840,16.032064,9.018036,1.402806,8.517034,0.501002,72.645291,0.200401,10.921844,20.440882,23.246493,0.901804,-1.603206,6.412826,-65.731463,6.713427,-4.008016,-13.527054,-16.332665,6.012024,-7.915832,0.100200,-72.044088,0.400802,-10.320641,-19.839679,-22.645291,-0.300601,32.865731,40.881764,-31.262525,41.182365,30.460922,20.941884,18.136273,40.480962,...,1,1,1,0,-1,-1,-1,-1,0,1,1,2,2,-1,-1,0,0,-2,1,1,2,2,0,2,1,1,2,2,0,2,0,969.0,889.0,1250.0,1505.0,11,7,80.0,-255.0,4
3,8.216433,0.000000,71.442886,0.0,13.226453,27.655311,18.036072,0.601202,8.917836,0.000000,38.877756,0.000000,71.142285,26.853707,14.829659,1.903808,-0.701403,8.216433,-30.661323,8.216433,-62.925852,-18.637275,-6.613226,6.312625,-8.917836,0.000000,-38.877756,0.000000,-71.142285,-26.853707,-14.829659,-1.903808,62.525050,71.442886,32.565130,71.442886,0.300601,44.589178,56.613226,69.539078,...,0,1,0,1,0,-1,0,-1,0,1,0,1,1,0,-1,0,0,-1,1,0,1,1,0,1,1,0,1,1,0,1,0,1300.0,1303.0,1731.0,1479.0,8,17,-3.0,252.0,-9
4,9.965035,0.174825,75.174825,0.0,39.160839,30.944056,11.713287,1.748252,6.293706,0.699301,47.552448,0.000000,17.307692,18.531469,22.027972,1.398601,3.671329,9.265734,-37.587413,9.965035,-7.342657,-8.566434,-12.062937,8.566434,-6.118881,-0.524476,-47.377622,0.174825,-17.132867,-18.356643,-21.853147,-1.223776,68.881119,74.475524,27.622378,75.174825,57.867133,56.643357,53.146853,73.776224,...,-1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1366.0,701.0,2130.0,1278.0,14,2,665.0,852.0,12


In [ ]:
train_list = x_train.columns.tolist()

In [ ]:
train_list

['P0_Ability_time',
 'P0_AddToControlGroup_time',
 'P0_Camera_time',
 'P0_ControlGroup_time',
 'P0_GetControlGroup_time',
 'P0_Right Click_time',
 'P0_Selection_time',
 'P0_SetControlGroup_time',
 'P1_Ability_time',
 'P1_AddToControlGroup_time',
 'P1_Camera_time',
 'P1_ControlGroup_time',
 'P1_GetControlGroup_time',
 'P1_Right Click_time',
 'P1_Selection_time',
 'P1_SetControlGroup_time',
 'diff_base110_time',
 'diff_base111_time',
 'diff_base112_time',
 'diff_base113_time',
 'diff_base114_time',
 'diff_base115_time',
 'diff_base116_time',
 'diff_base117_time',
 'diff_base210_time',
 'diff_base211_time',
 'diff_base212_time',
 'diff_base213_time',
 'diff_base214_time',
 'diff_base215_time',
 'diff_base216_time',
 'diff_base217_time',
 'diff_base310_time',
 'diff_base311_time',
 'diff_base312_time',
 'diff_base313_time',
 'diff_base314_time',
 'diff_base315_time',
 'diff_base316_time',
 'diff_base317_time',
 'diff_base410_time',
 'diff_base411_time',
 'diff_base412_time',
 'diff_base413

In [ ]:
x_test = x_test[train_list]

In [ ]:
x_test.head()

,P0_Ability_time,P0_AddToControlGroup_time,P0_Camera_time,P0_ControlGroup_time,P0_GetControlGroup_time,P0_Right Click_time,P0_Selection_time,P0_SetControlGroup_time,P1_Ability_time,P1_AddToControlGroup_time,P1_Camera_time,P1_ControlGroup_time,P1_GetControlGroup_time,P1_Right Click_time,P1_Selection_time,P1_SetControlGroup_time,diff_base110_time,diff_base111_time,diff_base112_time,diff_base113_time,diff_base114_time,diff_base115_time,diff_base116_time,diff_base117_time,diff_base210_time,diff_base211_time,diff_base212_time,diff_base213_time,diff_base214_time,diff_base215_time,diff_base216_time,diff_base217_time,diff_base310_time,diff_base311_time,diff_base312_time,diff_base313_time,diff_base314_time,diff_base315_time,diff_base316_time,diff_base317_time,...,diff_upgrade1610,diff_upgrade1612,diff_upgrade1614,diff_upgrade53,diff_upgrade73,diff_upgrade75,diff_upgrade93,diff_upgrade95,diff_upgrade97,diff_upgrade113,diff_upgrade115,diff_upgrade117,diff_upgrade119,diff_upgrade133,diff_upgrade135,diff_upgrade137,diff_upgrade139,diff_upgrade1311,diff_upgrade153,diff_upgrade155,diff_upgrade157,diff_upgrade159,diff_upgrade1511,diff_upgrade1513,diff_upgrade173,diff_upgrade175,diff_upgrade177,diff_upgrade179,diff_upgrade1711,diff_upgrade1713,diff_upgrade1715,P0_camera_var,P1_camera_var,P0_right_click_var,P1_right_click_var,P0_attack_count,P1_attack_count,diff_camera_var,diff_right_click_var,diff_attack_count
0,4.366197,0.140845,32.676056,0.000000,1.267606,33.943662,7.323944,0.704225,5.352113,0.563380,65.774648,0.0,6.056338,17.183099,5.774648,0.000000,-0.985915,3.802817,-61.408451,4.366197,-1.690141,-12.816901,-1.408451,4.366197,-5.211268,-0.422535,-65.633803,0.140845,-5.915493,-17.042254,-5.633803,0.140845,27.323944,32.112676,-33.098592,32.676056,26.619718,15.492958,26.901408,32.676056,...,-1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,635.0,1595.0,1312.0,1529.0,5,2,-960.0,-217.0,3
1,5.555556,0.000000,32.679739,0.653595,2.941176,8.333333,18.627451,0.000000,4.575163,0.163399,23.856209,0.0,19.934641,11.111111,3.104575,0.816993,0.980392,5.392157,-18.300654,5.555556,-14.379085,-5.555556,2.450980,4.738562,-4.575163,-0.163399,-23.856209,0.000000,-19.934641,-11.111111,-3.104575,-0.816993,28.104575,32.516340,8.823529,32.679739,12.745098,21.568627,29.575163,31.862745,...,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,984.0,1028.0,841.0,1744.0,0,0,-44.0,-903.0,0
2,7.051282,0.000000,52.350427,0.000000,13.461538,16.880342,8.974359,1.495726,6.196581,0.000000,47.008547,0.0,91.025641,23.076923,7.692308,1.068376,0.854701,7.051282,-39.957265,7.051282,-83.974359,-16.025641,-0.641026,5.982906,-6.196581,0.000000,-47.008547,0.000000,-91.025641,-23.076923,-7.692308,-1.068376,46.153846,52.350427,5.341880,52.350427,-38.675214,29.273504,44.658120,51.282051,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1739.0,451.0,2140.0,640.0,9,4,1288.0,1500.0,5
3,11.919699,1.003764,64.617315,0.000000,31.994981,27.352572,15.432873,1.129235,7.277290,0.125471,44.792974,0.0,3.011292,40.401506,12.421581,0.250941,4.642409,11.794228,-32.873275,11.919699,8.908407,-28.481807,-0.501882,11.668758,-6.273526,0.878294,-43.789210,1.003764,-2.007528,-39.397742,-11.417817,0.752823,57.340025,64.491844,19.824341,64.617315,61.606023,24.215809,52.195734,64.366374,...,0,1,0,0,0,0,0,0,0,2,2,2,2,0,0,0,0,-2,0,0,0,0,-2,0,0,0,0,0,-2,0,0,1903.0,1052.0,2720.0,2128.0,13,17,851.0,592.0,-4
4,2.473498,0.706714,45.583039,0.000000,136.749117,10.600707,8.833922,0.706714,2.826855,0.000000,56.890459,0.0,0.353357,2.473498,12.367491,0.353357,-0.353357,2.473498,-54.416961,2.473498,2.120141,0.000000,-9.893993,2.120141,-2.120141,0.706714,-56.183746,0.706714,0.353357,-1.766784,-11.660777,0.353357,42.756184,45.583039,-11.307420,45.583039,45.229682,43.109541,33.215548,45.229682,...,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,170.0,55.0,465.0,24.0,0,0,115.0,441.0,0


In [ ]:
x_test = x_test.fillna(0)

In [ ]:
x_test

,P0_species,P0_GetControlGroup,P0_Right Click,P1_species,time,P1_camera_var,P0_right_click_var,P1_right_click_var,game_id_P0_Ability_diff,game_id_P0_AddToControlGroup_diff,game_id_P0_Camera_diff,game_id_P0_ControlGroup_diff,game_id_P0_GetControlGroup_diff,game_id_P0_Right Click_diff,game_id_P0_Selection_diff,game_id_P0_SetControlGroup_diff,game_id_P1_Ability_diff,game_id_P1_AddToControlGroup_diff,game_id_P1_Camera_diff,game_id_P1_ControlGroup_diff,game_id_P1_GetControlGroup_diff,game_id_P1_Right Click_diff,game_id_P1_Selection_diff,game_id_P1_SetControlGroup_diff,game_id_time_diff,game_id_P0_total_unit_count_diff,game_id_P1_total_unit_count_diff,game_id_P0_worker_unit_count_diff,game_id_P1_worker_unit_count_diff,game_id_P0_Complex_unit_count_diff,game_id_P1_Complex_unit_count_diff,game_id_P0_A_unit_count_diff,game_id_P1_A_unit_count_diff,game_id_P0_B_unit_count_diff,game_id_P1_B_unit_count_diff,game_id_P0_C_unit_count_diff,game_id_P1_C_unit_count_diff,game_id_P0_D_unit_count_diff,game_id_P1_D_unit_count_diff,game_id_P0_air_unit_count_diff,...,P0_ground_attack_upgrade_count_P1_attack_count_diff,P1_ground_attack_upgrade_count_P0_all_species_skill_count_diff,P1_ground_attack_upgrade_count_P0_upgrade_diff,P1_ground_attack_upgrade_count_P0_attack_count_diff,P1_ground_attack_upgrade_count_P1_attack_count_diff,P0_ground_defence_upgrade_count_P1_all_species_skill_count_diff,P0_ground_defence_upgrade_count_P1_air_upgrade_diff,P0_ground_defence_upgrade_count_P1_upgrade_diff,P0_ground_defence_upgrade_count_P0_attack_count_diff,P1_ground_defence_upgrade_count_P1_air_upgrade_diff,P1_ground_defence_upgrade_count_P0_upgrade_diff,P1_ground_defence_upgrade_count_P1_attack_count_diff,P1_air_attack_upgrade_count_P1_all_species_skill_count_diff,P1_air_attack_upgrade_count_P1_attack_count_diff,P1_air_defence_upgrade_count_P1_all_species_skill_count_diff,P0_all_species_skill_count_P1_all_species_skill_count_diff,P0_all_species_skill_count_P1_ground_upgrade_diff,P0_all_species_skill_count_P1_upgrade_diff,P0_all_species_skill_count_P1_camera_var_diff,P0_all_species_skill_count_P1_attack_count_diff,P1_all_species_skill_count_P0_ground_upgrade_diff,P1_all_species_skill_count_P0_upgrade_diff,P1_air_upgrade_P1_attack_count_diff,P0_ground_upgrade_P1_upgrade_diff,P0_ground_upgrade_P1_attack_count_diff,P1_ground_upgrade_P0_upgrade_diff,P1_ground_upgrade_P1_attack_count_diff,P0_upgrade_P1_upgrade_diff,P1_upgrade_P0_camera_var_diff,P1_upgrade_P1_attack_count_diff,P0_camera_var_P1_camera_var_diff,P0_camera_var_P0_right_click_var_diff,P0_camera_var_P1_right_click_var_diff,P0_camera_var_P0_attack_count_diff,P1_camera_var_P0_right_click_var_diff,P1_camera_var_P1_right_click_var_diff,P1_camera_var_P0_attack_count_diff,P0_right_click_var_P1_attack_count_diff,P1_right_click_var_P1_attack_count_diff,P0_attack_count_P1_attack_count_diff
0,1,9.0,241.0,1,7.10,1595.0,1312.0,1529.0,38841.0,38871.0,38640.0,38872.0,38863.0,38631.0,38820.0,38867.0,38834.0,38868.0,38405.0,38872.0,38829.0,38750.0,38831.0,38872.0,38864.90,38859,38856,38861,38860,38872,38872,38861,38860,38861,38860,38872,38872,38872,38872,38871,...,-2,-1,0,-5,-2,-1,0,0,-5,0,0,-2,-1,-2,-1,0,1,1,-1594.0,-1,1,1,-2,0,-2,0,-2,0,-635.0,-2,-960.0,-677.0,-894.0,630.0,283.0,66.0,1590.0,1310.0,1527.0,3
1,2,18.0,51.0,1,6.12,1028.0,841.0,1744.0,38839.0,38873.0,38673.0,38869.0,38855.0,38822.0,38759.0,38873.0,38845.0,38872.0,38727.0,38873.0,38751.0,38805.0,38854.0,38868.0,38866.88,38855,38856,38866,38860,38873,38873,38860,38860,38860,38860,38873,38873,38873,38873,38873,...,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,-1027.0,1,0,0,0,0,0,0,0,0,-984.0,0,-44.0,143.0,-760.0,984.0,187.0,-716.0,1028.0,841.0,1744.0,0
2,0,63.0,79.0,0,4.68,451.0,2140.0,640.0,38841.0,38874.0,38629.0,38874.0,38811.0,38795.0,38832.0,38867.0,38845.0,38874.0,38654.0,38874.0,38448.0,38766.0,38838.0,38869.0,38869.32,38862,38867,38865,38869,38874,38874,38865,38869,38865,38869,38873,38872,38874,38874,38873,...,-4,0,0,-9,-4,0,0,0,-9,0,0,-4,0,-4,0,0,0,0,-451.0,-4,0,0,-4

In [ ]:
x_train

,P0_Ability_time,P0_AddToControlGroup_time,P0_Camera_time,P0_ControlGroup_time,P0_GetControlGroup_time,P0_Right Click_time,P0_Selection_time,P0_SetControlGroup_time,P1_Ability_time,P1_AddToControlGroup_time,P1_Camera_time,P1_ControlGroup_time,P1_GetControlGroup_time,P1_Right Click_time,P1_Selection_time,P1_SetControlGroup_time,diff_base110_time,diff_base111_time,diff_base112_time,diff_base113_time,diff_base114_time,diff_base115_time,diff_base116_time,diff_base117_time,diff_base210_time,diff_base211_time,diff_base212_time,diff_base213_time,diff_base214_time,diff_base215_time,diff_base216_time,diff_base217_time,diff_base310_time,diff_base311_time,diff_base312_time,diff_base313_time,diff_base314_time,diff_base315_time,diff_base316_time,diff_base317_time,...,diff_upgrade1610,diff_upgrade1612,diff_upgrade1614,diff_upgrade53,diff_upgrade73,diff_upgrade75,diff_upgrade93,diff_upgrade95,diff_upgrade97,diff_upgrade113,diff_upgrade115,diff_upgrade117,diff_upgrade119,diff_upgrade133,diff_upgrade135,diff_upgrade137,diff_upgrade139,diff_upgrade1311,diff_upgrade153,diff_upgrade155,diff_upgrade157,diff_upgrade159,diff_upgrade1511,diff_upgrade1513,diff_upgrade173,diff_upgrade175,diff_upgrade177,diff_upgrade179,diff_upgrade1711,diff_upgrade1713,diff_upgrade1715,P0_camera_var,P1_camera_var,P0_right_click_var,P1_right_click_var,P0_attack_count,P1_attack_count,diff_camera_var,diff_right_click_var,diff_attack_count
0,4.594595,0.270270,60.000000,0.000000,3.243243,4.729730,6.756757,0.405405,4.594595,0.000000,57.432432,0.000000,0.405405,3.783784,7.702703,0.135135,0.000000,4.594595,-52.837838,4.594595,4.189189,0.810811,-3.108108,4.459459,-4.324324,0.270270,-57.162162,0.270270,-0.135135,-3.513514,-7.432432,0.135135,55.405405,60.000000,2.567568,60.000000,59.594595,56.216216,52.297297,59.864865,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,887.0,44.0,2367.0,77.0,0,3,843.0,2290.0,-3
1,7.715431,0.100200,62.825651,0.000000,16.232465,16.032064,18.637275,1.002004,6.713427,0.000000,85.971944,0.000000,3.106212,13.126253,11.623246,0.801603,1.002004,7.715431,-78.256513,7.715431,4.609218,-5.410822,-3.907816,6.913828,-6.613226,0.100200,-85.871743,0.100200,-3.006012,-13.026052,-11.523046,-0.701403,56.112224,62.825651,-23.146293,62.825651,59.719439,49.699399,51.202405,62.024048,...,-2,1,0,-1,-1,0,-1,0,0,0,1,1,1,-1,0,0,0,-1,0,1,1,1,0,1,0,1,1,1,0,1,0,750.0,1821.0,2099.0,2230.0,2,4,-1071.0,-131.0,-2
2,6.913828,0.601202,41.382766,0.000000,9.919840,16.032064,9.018036,1.402806,8.517034,0.501002,72.645291,0.200401,10.921844,20.440882,23.246493,0.901804,-1.603206,6.412826,-65.731463,6.713427,-4.008016,-13.527054,-16.332665,6.012024,-7.915832,0.100200,-72.044088,0.400802,-10.320641,-19.839679,-22.645291,-0.300601,32.865731,40.881764,-31.262525,41.182365,30.460922,20.941884,18.136273,40.480962,...,1,1,1,0,-1,-1,-1,-1,0,1,1,2,2,-1,-1,0,0,-2,1,1,2,2,0,2,1,1,2,2,0,2,0,969.0,889.0,1250.0,1505.0,11,7,80.0,-255.0,4
3,8.216433,0.000000,71.442886,0.000000,13.226453,27.655311,18.036072,0.601202,8.917836,0.000000,38.877756,0.000000,71.142285,26.853707,14.829659,1.903808,-0.701403,8.216433,-30.661323,8.216433,-62.925852,-18.637275,-6.613226,6.312625,-8.917836,0.000000,-38.877756,0.000000,-71.142285,-26.853707,-14.829659,-1.903808,62.525050,71.442886,32.565130,71.442886,0.300601,44.589178,56.613226,69.539078,...,0,1,0,1,0,-1,0,-1,0,1,0,1,1,0,-1,0,0,-1,1,0,1,1,0,1,1,0,1,1,0,1,0,1300.0,1303.0,1731.0,1479.0,8,17,-3.0,252.0,-9
4,9.965035,0.174825,75.174825,0.000000,39.160839,30.944056,11.713287,1.748252,6.293706,0.699301,47.552448,0.000000,17.307692,18.531469,22.027972,1.398601,3.671329,9.265734,-37.587413,9.965035,-7.342657,-8.566434,-12.062937,8.566434,-6.118881,-0.524476,-47.377622,0.174825,-17.132867,-18.356643,-21.853147,-1.223776,68.881119,74.475524,27.622378,75.174825,57.867133,56.643357,53.146853,73.776224,...,-1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,1366.0,701.0,2130.0,1278.0,14,2,665.0,852.0,12
...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
x_test = x_test.drop(x_test.columns[[0, 1]], axis='columns')

In [ ]:
x_test

,P0_Ability_time,P0_AddToControlGroup_time,P0_Camera_time,P0_ControlGroup_time,P0_GetControlGroup_time,P0_Right Click_time,P0_Selection_time,P0_SetControlGroup_time,P1_Ability_time,P1_AddToControlGroup_time,P1_Camera_time,P1_ControlGroup_time,P1_GetControlGroup_time,P1_Right Click_time,P1_Selection_time,P1_SetControlGroup_time,diff_base110_time,diff_base111_time,diff_base112_time,diff_base113_time,diff_base114_time,diff_base115_time,diff_base116_time,diff_base117_time,diff_base210_time,diff_base211_time,diff_base212_time,diff_base213_time,diff_base214_time,diff_base215_time,diff_base216_time,diff_base217_time,diff_base310_time,diff_base311_time,diff_base312_time,diff_base313_time,diff_base314_time,diff_base315_time,diff_base316_time,diff_base317_time,...,diff_upgrade1610,diff_upgrade1612,diff_upgrade1614,diff_upgrade53,diff_upgrade73,diff_upgrade75,diff_upgrade93,diff_upgrade95,diff_upgrade97,diff_upgrade113,diff_upgrade115,diff_upgrade117,diff_upgrade119,diff_upgrade133,diff_upgrade135,diff_upgrade137,diff_upgrade139,diff_upgrade1311,diff_upgrade153,diff_upgrade155,diff_upgrade157,diff_upgrade159,diff_upgrade1511,diff_upgrade1513,diff_upgrade173,diff_upgrade175,diff_upgrade177,diff_upgrade179,diff_upgrade1711,diff_upgrade1713,diff_upgrade1715,P0_camera_var,P1_camera_var,P0_right_click_var,P1_right_click_var,P0_attack_count,P1_attack_count,diff_camera_var,diff_right_click_var,diff_attack_count
0,4.366197,0.140845,32.676056,0.000000,1.267606,33.943662,7.323944,0.704225,5.352113,0.563380,65.774648,0.0000,6.056338,17.183099,5.774648,0.000000,-0.985915,3.802817,-61.408451,4.366197,-1.690141,-12.816901,-1.408451,4.366197,-5.211268,-0.422535,-65.633803,0.140845,-5.915493,-17.042254,-5.633803,0.140845,27.323944,32.112676,-33.098592,32.676056,26.619718,15.492958,26.901408,32.676056,...,-1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,635.0,1595.0,1312.0,1529.0,5,2,-960.0,-217.0,3
1,5.555556,0.000000,32.679739,0.653595,2.941176,8.333333,18.627451,0.000000,4.575163,0.163399,23.856209,0.0000,19.934641,11.111111,3.104575,0.816993,0.980392,5.392157,-18.300654,5.555556,-14.379085,-5.555556,2.450980,4.738562,-4.575163,-0.163399,-23.856209,0.000000,-19.934641,-11.111111,-3.104575,-0.816993,28.104575,32.516340,8.823529,32.679739,12.745098,21.568627,29.575163,31.862745,...,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,984.0,1028.0,841.0,1744.0,0,0,-44.0,-903.0,0
2,7.051282,0.000000,52.350427,0.000000,13.461538,16.880342,8.974359,1.495726,6.196581,0.000000,47.008547,0.0000,91.025641,23.076923,7.692308,1.068376,0.854701,7.051282,-39.957265,7.051282,-83.974359,-16.025641,-0.641026,5.982906,-6.196581,0.000000,-47.008547,0.000000,-91.025641,-23.076923,-7.692308,-1.068376,46.153846,52.350427,5.341880,52.350427,-38.675214,29.273504,44.658120,51.282051,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1739.0,451.0,2140.0,640.0,9,4,1288.0,1500.0,5
3,11.919699,1.003764,64.617315,0.000000,31.994981,27.352572,15.432873,1.129235,7.277290,0.125471,44.792974,0.0000,3.011292,40.401506,12.421581,0.250941,4.642409,11.794228,-32.873275,11.919699,8.908407,-28.481807,-0.501882,11.668758,-6.273526,0.878294,-43.789210,1.003764,-2.007528,-39.397742,-11.417817,0.752823,57.340025,64.491844,19.824341,64.617315,61.606023,24.215809,52.195734,64.366374,...,0,1,0,0,0,0,0,0,0,2,2,2,2,0,0,0,0,-2,0,0,0,0,-2,0,0,0,0,0,-2,0,0,1903.0,1052.0,2720.0,2128.0,13,17,851.0,592.0,-4
4,2.473498,0.706714,45.583039,0.000000,136.749117,10.600707,8.833922,0.706714,2.826855,0.000000,56.890459,0.0000,0.353357,2.473498,12.367491,0.353357,-0.353357,2.473498,-54.416961,2.473498,2.120141,0.000000,-9.893993,2.120141,-2.120141,0.706714,-56.183746,0.706714,0.353357,-1.766784,-11.660777,0.353357,42.756184,45.583039,-11.307420,45.583039,45.229682,43.109541,33.215548,45.229682,...,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,0,0,-1,0,0,170.0,55.0,465.0,24.0,0,0,115.0,441.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
preds = []
for model in models:
    pred = model.predict_proba(x_test)[:, 1]
    preds.append(pred)
pred = np.mean(preds, axis=0)

submission = pd.read_csv('/content/drive/My Drive/data/sample_submission.csv', index_col=0)
submission['winner'] = submission['winner'] + pred
submission.to_csv('/content/drive/My Drive/data/submission_last.csv')
submission.head()

,winner
game_id,
38872,0.648883
38873,0.490051
38874,0.415552
38875,0.150411
38876,0.525769
